【NLP最佳实践】Huggingface Transformers实战教程：

https://www.heywhale.com/home/activity/detail/61dd2a3dc238c000186ac330

# 基于 ALBERT 进行句子相似性判断

- 二分类
- 回归

## 句子相似度计算的几种方法
1.句子相似度介绍：
句子相似度–指的是两个句子之间相似的程度。在NLP中有很大的用处，譬如对话系统，文本分类、信息检索、语义分析等，它可以为我们提供检索信息更快的方式，并且得到的信息更加准确。

2.句子相似计算的方法概括：
句子相似度计算主要分为：

基于统计的方法：
    莱文斯坦距离（编辑距离）

    BM25

    TFIDF计算

    TextRank算法中的句子相似性

基于深度学习的方法：

    基于Word2Vec的余弦相似度

    DSSM（Deep Structured Semantic Models）


## sentence-similarity

GitHub：https://github.com/yanqiangmiffy/sentence-similarity

问题句子相似度计算，即给定客服里用户描述的两句话，用算法来判断是否表示了相同的语义。
## 句子相似度判定
今年和去年前后相继出现了多个关于句子相似度判定的比赛，即得定两个句子，用算法判断是否表示了相同的语义或者意思。
`其中第4、5这个2个比赛数据集格式比较像，请见` [sent_match](https://github.com/yanqiangmiffy/sent_match),`，2、3 的数据集格式比较像，本仓库基于2、3数据集做实验`

下面是比赛的列表：
- 1 [Quora Question Pairs](https://www.kaggle.com/c/quora-question-pairs)

> The goal of this competition is to predict which of the provided pairs of questions contain two questions with the same meaning. 

> [数据集](https://www.kaggle.com/c/quora-question-pairs/data)未经过脱敏处理,用真实的英文单词标识

- 2 [ ATEC学习赛：NLP之问题相似度计算](https://dc.cloud.alipay.com/index#/topic/intro?id=8)
> 问题相似度计算，即给定客服里用户描述的两句话，用算法来判断是否表示了相同的语义。

> [数据集](https://dc.cloud.alipay.com/index#/topic/data?id=8)未经过脱敏处理
> 示例：
> 1. “花呗如何还款” --“花呗怎么还款”：同义问句
> 2. “花呗如何还款” -- “我怎么还我的花被呢”：同义问句
> 3. “花呗分期后逾期了如何还款”-- “花呗分期后逾期了哪里还款”：非同义问句
> 对于例子a，比较简单的方法就可以判定同义；对于例子b，包含了错别字、同义词、词序变换等问题，两个句子乍一看并不类似，想正确判断比较有挑战；对于例子c，两句> 话很类似，仅仅有一处细微的差别 “如何”和“哪里”，就导致语义不一致。"""

- 3 [CCKS 2018 微众银行智能客服问句匹配大赛](https://biendata.com/competition/CCKS2018_3/leaderboard/)
> 与基于Quora的的的语义等价判别相同，本次评测任务的主要目标是针对中文的真实客服语料，进行问句意图匹配。集给定两个语句，要求判定两者意图是否相同或者相近。所有语料来自原始的银行领域智能客服日志，并经过了筛选和人工的意图匹配标注。

>  [数据集](https://biendata.com/competition/CCKS2018_3/data/)经过脱敏处理
> 输入：一般几天能通过审核\ t一般审核通过要多久
> 输出：1

- 4 [CHIP 2018-第四届中国健康信息处理会议（CHIP）](https://biendata.com/competition/chip2018/)
> 本次评测任务的主要目标是针对中文的真实患者健康咨询语料，进行问句意图匹配。给定两个语句，要求判定两者意图是否相同或者相近。所有语料来自互联网上患者真实> 的问题，并经过了筛选和人工的意图匹配标注。平安云将为报名的队伍提供GPU的训练环境。

>  [数据集](https://biendata.com/competition/chip2018/data/)经过脱敏处理，问题由数字标示
> 训练集包含20000条左右标注好的数据（经过脱敏处理，包含标点符号），供参赛人员进行训练和测试。 测试集包含10000条左右无label的数据（经过脱敏处理，包含标点> 符号）。选手需要对测试集数据的label进行预测并提交。测试集数据作为AB榜的评测依据。

- 5 [第三届魔镜杯大赛](https://ai.ppdai.com/mirror/goToMirrorDetail?mirrorId=1)
> 智能客服聊天机器人场景中，待客户提出问题后，往往需要先计算客户提出问题与知识库问题的相似度，进而定位最相似问题，再对问题给出答案。本次比赛的题目便是问 > 题相似度算法设计。

>  [数据集](https://ai.ppdai.com/mirror/goToMirrorDetail?mirrorId=1)经过脱敏处理，问题由数字标示
>  为保护用户隐私并保证比赛的公平公正，所有原始文本信息都被编码成单字ID序列和词语ID序列。单字包含单个汉字、英文字母、标点及空格等；词语包含切词后的中> 文词语、英文单词、标点及空格等。单字ID和词语ID存在于两个不同的命名空间，即词语中的单字词或者标点，和单字中的相同字符及相同标点不一定有同一个ID。其> > 中，单字序列以L开头，词语序列以W开头。


## 实验简介

我们将在本笔记本中学习如何为**句子对分类**任务微调 ALBERT 和其他基于 BERT 的模型。这个 PyTorch 实现利用 Hugging face *transformers* 和 *datasets* 库来下载预训练模型、实现快速研究实验、访问数据集和评估指标。

该任务是语义文本相似性问题的一部分。你有两对句子，你想对它们之间的文本交互进行建模。

本笔记本中使用的数据集是 Microsoft Research Paraphrase Corpus (MRPC)，它是 GLUE 基准测试的一部分：您有两个句子，您想预测一个句子是否是另一个句子的释义。评价指标为 F1 和准确率。

您应该能够在验证集上达到 **91.19** 作为 F1 分数（ALBERT 论文中报告的分数是 90.9）和 **87.5** 作为准确度。微调每个 epoch 需要 35 秒，推理需要 2 秒。



GLUE共有九个任务，分别是CoLA、SST-2、MRPC、STS-B、QQP、MNLI、QNLI、RTE、WNLI。可以分为三类，分别是单句任务，相似性和释义任务:
    
- 单句任务：CoLA、SST-2
- 相似性任务：MRPC、STS-B、QQP
- 释义任务：MNLI、QNLI、RTE、WNLI

通常来说，NLP可以分为自然语言理解（NLU）和自然语言生成（NLG）。在NLU方面，我们拿时下最流行的GLUE(General Language Understanding Evaluation)排行榜举例，其上集合了九项NLU的任务，分别是

- CoLA(The Corpus of Linguistic Acceptability):纽约大学发布的有关语法的数据集，该任务主要是对一个给定句子，判定其是否语法正确，因此CoLA属于单个句子的文本二分类任务；
- SST(The Stanford Sentiment Treebank)，是斯坦福大学发布的一个情感分析数据集，主要针对电影评论来做情感分类，因此SST属于单个句子的文本分类任务（其中SST-2是二分类，SST-5是五分类，SST-5的情感极性区分的更细致）；
- MRPC(Microsoft Research Paraphrase Corpus)，由微软发布，判断两个给定句子，是否具有相同的语义，属于句子对的文本二分类任务；
- STS-B(Semantic Textual Similarity Benchmark)，主要是来自于历年SemEval中的一个任务（同时该数据集也包含在了SentEval），具体来说是用1到5的分数来表征两个句子的语义相似性，本质上是一个回归问题，但依然可以用分类的方法做，因此可以归类为句子对的文本五分类任务；
- QQP(Quora Question Pairs)，是由Quora发布的两个句子是否语义一致的数据集，属于句子对的文本二分类任务；
- MNLI(Multi-Genre Natural Language Inference)，同样由纽约大学发布，是一个文本蕴含的任务，在给定前提（Premise）下，需要判断假设（Hypothesis）是否成立，其中因为MNLI主打卖点是集合了许多不同领域风格的文本，因此又分为matched和mismatched两个版本的MNLI数据集，前者指训练集和测试集的数据来源一致，而后者指来源不一致。该任务属于句子对的文本三分类问题。
- QNLI（Question Natural Language Inference)，其前身是SQuAD 1.0数据集，给定一个问句，需要判断给定文本中是否包含该问句的正确答案。属于句子对的文本二分类任务；
- RTE(Recognizing Textual Entailment)，和MNLI类似，也是一个文本蕴含任务，不同的是MNLI是三分类，RTE只需要判断两个句子是否能够推断或对齐，属于句子对的文本二分类任务；
- WNLI(Winograd Natural Language Inference)，也是一个文本蕴含任务，不过似乎GLUE上这个数据集还有些问题；

In [ ]:
# 中文语言理解测评基准(CLUE)
# https://www.cluebenchmarks.com/classification.html

本教程的主要特点是：
- 端到端 ML 实施（训练、验证、预测、评估）
- 轻松适应您自己的数据集
- 促进其他基于 BERT 的模型（BERT、ALBERT、...）的快速实验
- 使用有限的计算资源进行快速训练（混合精度、梯度累积……）
- 多 GPU 执行
- 分类决策的阈值选择（不一定是 0.5）
- 冻结 BERT 层，只更新分类层权重或更新所有权重
- 种子设置，可复现结果

#### 目录

1. [安装库和导入](#section01)

2. [加载数据集](#section02)

3. [类和函数](#section03)

4. [参数](#section04)

5. [训练与验证](#section05)

6. [预测](#section06)

7. [评价](#section07)

8. [实验思路](#section08)


## 安装并导入所需要的库

In [1]:
# !pip install datasets
# !pip install transformers

In [15]:
import torch
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import copy
import torch.optim as optim
import random
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup
from datasets import load_dataset, load_metric

os.environ["TOKENIZERS_PARALLELISM"] = "false" # 

In [3]:
# 查看运行资源信息
# !pip -q install gputil
# !pip -q install psutil
# !pip -q install humanize

psutil可以用来做系统监控，性能分析，进程管理。 支持的系统有Linux, Windows, OSX, FreeBSD and Sun Solaris，32和64位系统都支持，同时支持pyhton2.4到3.4。

In [16]:
# !/usr/bin/env python
# -*- coding: utf-8 -*-
import psutil
import datetime
import time
 
# 当前时间
now_time = time.strftime('%Y-%m-%d-%H:%M:%S', time.localtime(time.time()))
print(now_time)
# 查看cpu的信息
print ("物理CPU个数: %s" % psutil.cpu_count(logical=False))
cpu = (str)(psutil.cpu_percent(1)) + '%'
print (u"cup使用率: %s" % cpu)
# 查看内存信息,剩余内存.free  总共.total
free = str(round(psutil.virtual_memory().free/(1024.0*1024.0*1024.0), 2))
total = str(round(psutil.virtual_memory().total/(1024.0*1024.0*1024.0), 2))
memory = int(psutil.virtual_memory().total-psutil.virtual_memory().free)/float(psutil.virtual_memory().total)
print(u"物理内存： %s G" % total)
print (u"剩余物理内存： %s G" % free)
print (u"物理内存使用率： %s %%" % int(memory*100))
# 系统启动时间
print (u"系统启动时间: %s" % datetime.datetime.fromtimestamp(psutil.boot_time()).strftime("%Y-%m-%d %H:%M:%S"))
# 系统用户
users_count = len(psutil.users())
users_list = ",".join([u.name for u in psutil.users()])
print (u"当前有%s个用户，分别是 %s" % (users_count, users_list))
# 网卡，可以得到网卡属性，连接数，当前流量等信息
net = psutil.net_io_counters()
bytes_sent = '{0:.2f} Mb'.format(net.bytes_recv / 1024/1024)
bytes_rcvd = '{0:.2f} Mb'.format(net.bytes_sent / 1024/1024)
print (u"网卡接收流量 %s 网卡发送流量 %s" % (bytes_rcvd, bytes_sent))
io = psutil.disk_partitions()
del io[-1]
print ('-----------------------------磁盘信息---------------------------------------')
print ("系统磁盘信息："+str(io))
for i in io:
    o = psutil.disk_usage(i.device)
    print ("总容量："+str(int(o.total/(1024.0*1024.0*1024.0)))+"G")
    print ("已用容量："+str(int(o.used/(1024.0*1024.0*1024.0)))+"G")
    print ("可用容量："+str(int(o.free/(1024.0*1024.0*1024.0)))+"G")
# print('-----------------------------进程信息-------------------------------------')
# # 查看系统全部进程
# for pnum in psutil.pids():
#     p = psutil.Process(pnum)
#     print ("进程名 %-20s  内存利用率 %-18s 进程状态 %-10s 创建时间 %-10s "% (p.name(), p.memory_percent(), p.status(),  p.create_time()))

2022-03-30-21:15:28
物理CPU个数: 8
cup使用率: 93.2%
物理内存： 31.95 G
剩余物理内存： 15.28 G
物理内存使用率： 52 %
系统启动时间: 2022-03-26 23:34:30
当前有1个用户，分别是 yanqiang
网卡接收流量 4412.50 Mb 网卡发送流量 16697.36 Mb
-----------------------------磁盘信息---------------------------------------
系统磁盘信息：[sdiskpart(device='C:\\', mountpoint='C:\\', fstype='NTFS', opts='rw,fixed', maxfile=255, maxpath=260), sdiskpart(device='D:\\', mountpoint='D:\\', fstype='NTFS', opts='rw,fixed', maxfile=255, maxpath=260), sdiskpart(device='E:\\', mountpoint='E:\\', fstype='NTFS', opts='rw,fixed', maxfile=255, maxpath=260)]
总容量：464G
已用容量：318G
可用容量：145G
总容量：531G
已用容量：358G
可用容量：172G
总容量：399G
已用容量：259G
可用容量：140G


In [17]:
# Check that we are using 100% of GPU memory footprint support libraries/code
# from https://github.com/patrickvonplaten/notebooks/blob/master/PyTorch_Reformer.ipynb


import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 16.4 GB  | Proc size: 467.8 MB
GPU RAM Free: 14057MB | Used: 10282MB | Util  42% | Total 24576MB


## 加载数据集

### 加载方式1

In [18]:
# 加载mrpc数据集
dataset = load_dataset('glue', 'mrpc')

Using the latest cached version of the module from C:\Users\yanqiang\.cache\huggingface\modules\datasets_modules\datasets\glue\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad (last modified on Thu Mar 24 22:33:16 2022) since it couldn't be found locally at glue., or remotely on the Hugging Face Hub.
Reusing dataset glue (C:\Users\yanqiang\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

In [21]:
# dataset

In [22]:
# 将数据保存为DataFrame格式，并保存成csv格式
dataset['train'].to_pandas().to_csv('data/04/train.csv',index=None)
dataset['test'].to_pandas().to_csv('data/04/test.csv',index=None)
dataset['validation'].to_pandas().to_csv('data/04/validation.csv',index=None)

In [23]:
# 将数据转为DataFrame格式
df_train = dataset['train'].to_pandas()
df_val = dataset['test'].to_pandas()
df_test = dataset['validation'].to_pandas()

如果想使用自己的数据集，可以将其上传到笔记本左侧。

目前只处理 csv 文件，您需要上传三个文件：训练数据、验证数据和测试数据（带或不带标签）

这是一个从 csv 文件加载数据的脚本，其标题如下：
- 句子1
- 句子2
- 标签

In [25]:
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(3668, 4)
(1725, 4)
(408, 4)


In [9]:
df_train.head()

,sentence1,sentence2,label,idx
0,"Amrozi accused his brother , whom he called "" ...","Referring to him as only "" the witness "" , Amr...",1,0
1,Yucaipa owned Dominick 's before selling the c...,Yucaipa bought Dominick 's in 1995 for $ 693 m...,0,1
2,They had published an advertisement on the Int...,"On June 10 , the ship 's owners had published ...",1,2
3,"Around 0335 GMT , Tab shares were up 19 cents ...","Tab shares jumped 20 cents , or 4.6 % , to set...",0,3
4,"The stock rose $ 2.11 , or about 11 percent , ...",PG & E Corp. shares jumped $ 1.63 or 8 percent...,1,4


### 加载方式2 直接读取CSV

In [ ]:
import pandas as pd
df_train = pd.read_csv('/home/mw/input/task048673/train.csv')
df_val = pd.read_csv('/home/mw/input/task048673/validation.csv')
df_test = pd.read_csv('/home/mw/input/task048673/test.csv')

## 数据集与模型定义

In [28]:
class CustomDataset(Dataset):

    def __init__(self, data, maxlen, with_labels=True, bert_model='albert-base-v2'):

        self.data = data  # pandas dataframe
        #Initialize the tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(bert_model)  

        self.maxlen = maxlen
        self.with_labels = with_labels 

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):

        #根据索引索取DataFrame中句子1余句子2
        sent1 = str(self.data.loc[index, 'sentence1'])
        sent2 = str(self.data.loc[index, 'sentence2'])

        # 对句子对分词，得到input_ids、attention_mask和token_type_ids
        encoded_pair = self.tokenizer(sent1, sent2, 
                                      padding='max_length',  # 填充到最大长度
                                      truncation=True,  # 根据最大长度进行截断
                                      max_length=self.maxlen,  
                                      return_tensors='pt')  # 返回torch.Tensor张量
        
        token_ids = encoded_pair['input_ids'].squeeze(0)  # tensor token ids
        attn_masks = encoded_pair['attention_mask'].squeeze(0)  # padded values对应为 "0" ，其他token为1
        token_type_ids = encoded_pair['token_type_ids'].squeeze(0)  #第一个句子的值为0，第二个句子的值为1 # 只有一句全为0

        if self.with_labels:  # True if the dataset has labels
            label = self.data.loc[index, 'label']
            return token_ids, attn_masks, token_type_ids, label  
        else:
            return token_ids, attn_masks, token_type_ids

In [29]:
class SentencePairClassifier(nn.Module):

    def __init__(self, bert_model="albert-base-v2", freeze_bert=False):
        super(SentencePairClassifier, self).__init__()
        #  初始化预训练模型Bert xxx
        self.bert_layer = AutoModel.from_pretrained(bert_model)

        #  encoder 隐藏层大小
        if bert_model == "albert-base-v2":  # 12M 参数
            hidden_size = 768
        elif bert_model == "albert-large-v2":  # 18M 参数
            hidden_size = 1024
        elif bert_model == "albert-xlarge-v2":  # 60M 参数
            hidden_size = 2048
        elif bert_model == "albert-xxlarge-v2":  # 235M 参数
            hidden_size = 4096
        elif bert_model == "bert-base-uncased": # 110M 参数
            hidden_size = 768
        elif bert_model == "roberta-base": # 
            hidden_size = 768

        # 固定Bert层 更新分类输出层
        if freeze_bert:
            for p in self.bert_layer.parameters():
                p.requires_grad = False
                
        self.dropout = nn.Dropout(p=0.1)
        # 分类输出
        self.cls_layer = nn.Linear(hidden_size, 1)


    @autocast()  # 混合精度训练
    def forward(self, input_ids, attn_masks, token_type_ids):
        '''
        Inputs:
            -input_ids : Tensor  containing token ids
            -attn_masks : Tensor containing attention masks to be used to focus on non-padded values
            -token_type_ids : Tensor containing token type ids to be used to identify sentence1 and sentence2
        '''

        # 输入给Bert，获取上下文表示
        # cont_reps, pooler_output = self.bert_layer(input_ids, attn_masks, token_type_ids)
        outputs = self.bert_layer(input_ids, attn_masks, token_type_ids)
        # last_hidden_state,pooler_output,all_hidden_states 12层
        # 将last layer hidden-state of the [CLS] 输入到 classifier layer
        # - last_hidden_state 的向量平均
        # - 取all_hidden_states最后四层，然后做平均 weighted 平均
        # - last_hidden_state+lstm
        # 获取输出
        logits = self.cls_layer(self.dropout(outputs['pooler_output']))

        return logits

In [30]:
def set_seed(seed):
    """ 固定随机种子，保证结果复现
    """
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    

def evaluate_loss(net, device, criterion, dataloader):
    """
    评估输出
    """
    net.eval()

    mean_loss = 0
    count = 0

    with torch.no_grad():
        for it, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm(dataloader)):
            seq, attn_masks, token_type_ids, labels = \
                seq.to(device), attn_masks.to(device), token_type_ids.to(device), labels.to(device)
            logits = net(seq, attn_masks, token_type_ids)
            mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            count += 1

    return mean_loss / count

In [31]:
print("创建models目录")
!mkdir models

创建models目录


子目录或文件 models 已经存在。


In [32]:
# bs=2
# accumu_step=2

# =>bs=2*2=4

In [33]:
def train_bert(net, criterion, opti, lr, lr_scheduler, train_loader, val_loader, epochs, iters_to_accumulate):

    best_loss = np.Inf
    best_ep = 1
    nb_iterations = len(train_loader)
    print_every = nb_iterations // 5  # 打印频率
    iters = []
    train_losses = []
    val_losses = []

    scaler = GradScaler()

    for ep in range(epochs):

        net.train()
        running_loss = 0.0
        for it, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm(train_loader)):

            # 转为cuda张量
            seq, attn_masks, token_type_ids, labels = \
                seq.to(device), attn_masks.to(device), token_type_ids.to(device), labels.to(device)
    
            # 混合精度加速训练
            with autocast():
                # Obtaining the logits from the model
                logits = net(seq, attn_masks, token_type_ids)

                # Computing loss
                loss = criterion(logits.squeeze(-1), labels.float())
                loss = loss / iters_to_accumulate  # Normalize the loss because it is averaged

            # Backpropagating the gradients
            # Scales loss.  Calls backward() on scaled loss to create scaled gradients.
            scaler.scale(loss).backward()

            if (it + 1) % iters_to_accumulate == 0:
                # Optimization step
                # scaler.step() first unscales the gradients of the optimizer's assigned params.
                # If these gradients do not contain infs or NaNs, opti.step() is then called,
                # otherwise, opti.step() is skipped.
                scaler.step(opti)
                # Updates the scale for next iteration.
                scaler.update()
                # 根据迭代次数调整学习率。
                lr_scheduler.step()
                # 梯度清零
                opti.zero_grad()


            running_loss += loss.item()

            if (it + 1) % print_every == 0:  # Print training loss information
                print()
                print("Iteration {}/{} of epoch {} complete. Loss : {} "
                      .format(it+1, nb_iterations, ep+1, running_loss / print_every))

                running_loss = 0.0


        val_loss = evaluate_loss(net, device, criterion, val_loader)  # Compute validation loss
        print()
        print("Epoch {} complete! Validation Loss : {}".format(ep+1, val_loss))

        if val_loss < best_loss:
            print("Best validation loss improved from {} to {}".format(best_loss, val_loss))
            print()
            net_copy = copy.deepcopy(net)  # # 保存最优模型
            best_loss = val_loss
            best_ep = ep + 1

    # 保存模型
    path_to_model='models/{}_lr_{}_val_loss_{}_ep_{}.pt'.format(bert_model, lr, round(best_loss, 5), best_ep)
    torch.save(net_copy.state_dict(), path_to_model)
    print("The model has been saved in {}".format(path_to_model))

    del loss
    torch.cuda.empty_cache() # 清空显存

## 超参数设置

In [34]:
bert_model = "albert-base-v2"  # 'albert-base-v2', 'albert-large-v2', 'albert-xlarge-v2', 'albert-xxlarge-v2', 'bert-base-uncased', ...
freeze_bert = False  # 是否冻结Bert
maxlen = 128  # 最大长度
bs = 16  # batch size
iters_to_accumulate = 2  # 梯度累加
lr = 2e-5  # learning rate
epochs = 4  # 训练轮数

## 训练与评估

AdamW优化器介绍 :
https://huggingface.co/transformers/main_classes/optimizer_schedules.html

In [35]:
#  固定随机种子 便于复现
set_seed(1) # 2022 

# 创建训练集与验证集
print("Reading training data...")
train_set = CustomDataset(df_train, maxlen, bert_model)
print("Reading validation data...")
val_set = CustomDataset(df_val, maxlen, bert_model)
# 常见训练集与验证集DataLoader
train_loader = DataLoader(train_set, batch_size=bs, num_workers=0)
val_loader = DataLoader(val_set, batch_size=bs, num_workers=0)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = SentencePairClassifier(bert_model, freeze_bert=freeze_bert)

if torch.cuda.device_count() > 1:  # if multiple GPUs
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    net = nn.DataParallel(net)

net.to(device)

criterion = nn.BCEWithLogitsLoss()
opti = AdamW(net.parameters(), lr=lr, weight_decay=1e-2)
num_warmup_steps = 0 # The number of steps for the warmup phase.
num_training_steps = epochs * len(train_loader)  # The total number of training steps
t_total = (len(train_loader) // iters_to_accumulate) * epochs  # Necessary to take into account Gradient accumulation
lr_scheduler = get_linear_schedule_with_warmup(optimizer=opti, num_warmup_steps=num_warmup_steps, num_training_steps=t_total)

train_bert(net, criterion, opti, lr, lr_scheduler, train_loader, val_loader, epochs, iters_to_accumulate)

Reading training data...
Reading validation data...


Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertModel: ['predictions.bias', 'predictions.decoder.bias', 'predictions.LayerNorm.bias', 'predictions.decoder.weight', 'predictions.dense.bias', 'predictions.dense.weight', 'predictions.LayerNorm.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
F:\ProgramData\Anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_dep


Iteration 46/230 of epoch 1 complete. Loss : 0.3110983663279077 


 40%|████████████████████████████████▊                                                | 93/230 [00:15<00:21,  6.40it/s]


Iteration 92/230 of epoch 1 complete. Loss : 0.2986520425133083 


...


Iteration 184/230 of epoch 4 complete. Loss : 0.05297895232417985 


100%|████████████████████████████████████████████████████████████████████████████████| 230/230 [00:35<00:00,  6.47it/s]



Iteration 230/230 of epoch 4 complete. Loss : 0.035685401976756424 


100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:06<00:00, 15.47it/s]



Epoch 4 complete! Validation Loss : 0.3996199344595273
The model has been saved in models/albert-base-v2_lr_2e-05_val_loss_0.347_ep_2.pt


<!-- 您可以通过浏览notebook左侧的文件下载保存在“models”文件夹中的模型 -->

## 预测

In [17]:
print("Creation of the results' folder...")
!mkdir results

Creation of the results' folder...


子目录或文件 results 已经存在。


In [ ]:
# [-0.1,0.1,-0.5]

In [18]:
def get_probs_from_logits(logits):
    """
    Converts a tensor of logits into an array of probabilities by applying the sigmoid function
    """
    probs = torch.sigmoid(logits.unsqueeze(-1))
    return probs.detach().cpu().numpy()

def test_prediction(net, device, dataloader, with_labels=True, result_file="results/output.txt"):
    """
    Predict the probabilities on a dataset with or without labels and print the result in a file
    """
    net.eval()
    w = open(result_file, 'w')
    probs_all = []

    with torch.no_grad():
        if with_labels:
            for seq, attn_masks, token_type_ids, _ in tqdm(dataloader):# 训练集、验证集
                seq, attn_masks, token_type_ids = seq.to(device), attn_masks.to(device), token_type_ids.to(device)
                logits = net(seq, attn_masks, token_type_ids)
                probs = get_probs_from_logits(logits.squeeze(-1)).squeeze(-1)
                probs_all += probs.tolist()
        else:
            for seq, attn_masks, token_type_ids in tqdm(dataloader): # 没有标签的测试集
                seq, attn_masks, token_type_ids = seq.to(device), attn_masks.to(device), token_type_ids.to(device)
                logits = net(seq, attn_masks, token_type_ids)
                probs = get_probs_from_logits(logits.squeeze(-1)).squeeze(-1)
                probs_all += probs.tolist()

    w.writelines(str(prob)+'\n' for prob in probs_all)
    w.close()

In [19]:
path_to_model = 'models/albert-base-v2_lr_2e-05_val_loss_0.347_ep_2.pt'  
# path_to_model = '/content/models/...'  # You can add here your trained model

path_to_output_file = 'results/output.txt'

print("Reading test data...")
test_set = CustomDataset(df_test, maxlen, bert_model)
test_loader = DataLoader(test_set, batch_size=bs, num_workers=0)

model = SentencePairClassifier(bert_model)
if torch.cuda.device_count() > 1:  # if multiple GPUs
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)

print()
print("Loading the weights of the model...")
model.load_state_dict(torch.load(path_to_model))
model.to(device)

print("Predicting on test data...")
test_prediction(net=model, device=device, dataloader=test_loader, with_labels=True,  # set the with_labels parameter to False if your want to get predictions on a dataset without labels
                result_file=path_to_output_file)
print()
print("Predictions are available in : {}".format(path_to_output_file))

Reading test data...


Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertModel: ['predictions.LayerNorm.bias', 'predictions.bias', 'predictions.decoder.weight', 'predictions.dense.bias', 'predictions.decoder.bias', 'predictions.dense.weight', 'predictions.LayerNorm.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Loading the weights of the model...
Predicting on test data...


100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [00:01<00:00, 19.23it/s]


Predictions are available in : results/output.txt


预测结果保存到results下面

## 评估

In [52]:
path_to_output_file = 'results/output.txt'  # 预测结果概率文件

labels_test = df_test['label']  # true labels

probs_test = pd.read_csv(path_to_output_file, header=None)[0]  # 预测概率
threshold = 0.6   # you can adjust this threshold for your own dataset
preds_test=(probs_test>=threshold).astype('uint8') # predicted labels using the above fixed threshold

# metric = load_metric("glue", "mrpc")

In [53]:
pd.read_csv(path_to_output_file, header=None)

,0
0,0.972656
1,0.153931
2,0.172852
3,0.881836
4,0.091248
...,...
403,0.063599
404,0.193604
405,0.986328
406,0.089111


In [54]:
preds_test

0      1
1      0
2      0
3      1
4      0
      ..
403    0
404    0
405    1
406    0
407    1
Name: 0, Length: 408, dtype: uint8

In [55]:
from sklearn.metrics import classification_report

In [56]:
print(classification_report(labels_test,preds_test))

              precision    recall  f1-score   support

           0       0.74      0.88      0.80       129
           1       0.94      0.86      0.90       279

    accuracy                           0.87       408
   macro avg       0.84      0.87      0.85       408
weighted avg       0.88      0.87      0.87       408



In [57]:
from sklearn.metrics import confusion_matrix

In [58]:
print(confusion_matrix(labels_test,preds_test))

[[113  16]
 [ 39 240]]


## 可以尝试的想法

- 尝试其他预训练模型：https://huggingface.co/models
- 尝试其他优化器和学习率调度器
- 调整超参数：批量大小、梯度累积参数 (iters_to_accumulate)、轮数、学习率
- 更改 *maxlen* 参数（最大值：512）。 如果增加它，训练将需要更长的时间
- 观察冻结编码器权重并仅更新分类器权重的影响
- 根据任务和数据集使用其他指标（Precision、Recall、ROC AUC、Precision-recall AUC 等）

## 进一步优化

- 调整代码，以便其他基于 BERT 的模型（如 RoBERTa 和 DistillBERT）也可以针对此任务进行微调
- 实验向分类层提供最后一层隐藏状态的所有输入标记的平均值或使用多个编码器层而不是池化器输出的其他操作